# 01 Project

*Louise Holst Andersen* | *The Lede Program* | *July, 2024*

**Snowfall.**

## Is climate change eating away at Tromsø's skiing season?
I want to find out if there is less snow in Tromsø, a town of 80,000 people in the Arctic part of Norway - and my home - than there used to be.

This notebook is only for retrieving and analysing **SNOWFALL** data from the [Frost API](https://frost.met.no/index.html) made available by *Meteorologisk institutt*, the Norwegian weather organisation.

The original notebook is called **01-project-snowdepth**.

### Initial setup

In [7]:
import requests
import pandas as pd

In [8]:
from dotenv import load_dotenv
import os

load_dotenv()

CLIENT_ID = os.getenv("CLIENT_ID")

In [9]:
# Let's put the endpoint into a variable
url = 'https://frost.met.no/observations/v0.jsonld'

### Get data

In [10]:
# Define the repeating time interval with no gaps for each year from 1988 to 2024
interval_start = "1988-10-01"
interval_end = "1989-06-01"
repetitions = 36  # number of repetitions
period = "P1Y"  # period of 1 year

referencetime = f"R{repetitions}/{interval_start}/{interval_end}/{period}"

# Set parametres for the API call
parametres = {
    'sources': 'SN90450', # Tromsø
    'elements': 'over_time(thickness_of_snowfall_amount P1D)', # this is the snowfall weather element
    'referencetime': referencetime,
}

# Get the endpoint, parametres and client ID
response = requests.get(url, parametres, auth=(CLIENT_ID,''))

# Convert the JSON to a dictionary
json = response.json()

# Check if it worked
if response.status_code == 200:
    data = json['data']
    print('Data retrieved from frost.met.no!')
else:
    print('Error! Returned status code %s' % response.status_code)
    print('Message: %s' % json['error']['message'])

Data retrieved from frost.met.no!


In [11]:
# Check the data
data[-1]

{'sourceId': 'SN90450:0',
 'referenceTime': '2024-05-31T00:00:00.000Z',
 'observations': [{'elementId': 'over_time(thickness_of_snowfall_amount P1D)',
   'value': 0,
   'unit': 'cm',
   'timeOffset': 'PT6H',
   'timeResolution': 'P1D',
   'timeSeriesId': 0,
   'performanceCategory': 'C',
   'exposureCategory': '2',
   'qualityCode': 2}]}

### Turn it into a DataFrame

In [12]:
# Initialize an empty list to store individual DataFrames
df_list = []

# Convert JSON data to a pandas DataFrame
for entry in data:
    row = pd.DataFrame(entry['observations'])
    row['referenceTime'] = entry['referenceTime']
    row['sourceId'] = entry['sourceId']
    df_list.append(row)

# Concatenate all DataFrames in the list
df = pd.concat(df_list, ignore_index=True)

In [13]:
df.head()

,elementId,value,unit,timeOffset,timeResolution,timeSeriesId,performanceCategory,exposureCategory,qualityCode,referenceTime,sourceId
0,over_time(thickness_of_snowfall_amount P1D),3,cm,PT6H,P1D,0,C,2,2,1988-10-23T00:00:00.000Z,SN90450:0
1,over_time(thickness_of_snowfall_amount P1D),8,cm,PT6H,P1D,0,C,2,2,1988-10-24T00:00:00.000Z,SN90450:0
2,over_time(thickness_of_snowfall_amount P1D),10,cm,PT6H,P1D,0,C,2,2,1988-10-25T00:00:00.000Z,SN90450:0
3,over_time(thickness_of_snowfall_amount P1D),11,cm,PT6H,P1D,0,C,2,2,1988-10-26T00:00:00.000Z,SN90450:0
4,over_time(thickness_of_snowfall_amount P1D),1,cm,PT6H,P1D,0,C,2,2,1988-10-27T00:00:00.000Z,SN90450:0


#### Remove some columns

In [14]:
# These additional columns will be kept
columns = ['sourceId','referenceTime','elementId','value','unit','timeOffset','timeResolution']
df2 = df[columns].copy()

# Convert the time value to something Python understands
df2['referenceTime'] = pd.to_datetime(df2['referenceTime'])

In [15]:
df2.head()

,sourceId,referenceTime,elementId,value,unit,timeOffset,timeResolution
0,SN90450:0,1988-10-23 00:00:00+00:00,over_time(thickness_of_snowfall_amount P1D),3,cm,PT6H,P1D
1,SN90450:0,1988-10-24 00:00:00+00:00,over_time(thickness_of_snowfall_amount P1D),8,cm,PT6H,P1D
2,SN90450:0,1988-10-25 00:00:00+00:00,over_time(thickness_of_snowfall_amount P1D),10,cm,PT6H,P1D
3,SN90450:0,1988-10-26 00:00:00+00:00,over_time(thickness_of_snowfall_amount P1D),11,cm,PT6H,P1D
4,SN90450:0,1988-10-27 00:00:00+00:00,over_time(thickness_of_snowfall_amount P1D),1,cm,PT6H,P1D


### Data analysis

#### Initial summary statistics

In [16]:
df2.describe()

,value
count,6977.000000
mean,2.406478
std,4.831852
min,0.000000
25%,0.000000
50%,0.000000
75%,2.000000
max,40.000000
